In [1]:
import pyWinhook as pyHook
import pythoncom
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
from IPython.display import clear_output
import csv
import sqlite3
import tkinter as tk

In [2]:
con = sqlite3.connect("key.db")
cur = con.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS KeyboardInput(user text, key integer, keyEvent char(10), Time integer)")
userName = ''

In [3]:
con2 = sqlite3.connect("key.db")
cur2 = con2.cursor()
cur2.execute("CREATE TABLE IF NOT EXISTS KeyboardInputVerification(user text, key integer, keyEvent char(10), Time integer)")
cur2.execute("DELETE FROM KeyboardInputVerification")
con2.commit()
userName = ''
ver = True

In [4]:
def userRecordData(eventList):
    if ver:
        sqlite_insert_query = """INSERT INTO KeyboardInputVerification
                              (user, key, keyEvent, Time) 
                              VALUES (?, ?, ?, ?);"""    
        cur2.executemany(sqlite_insert_query, eventList)
        con2.commit()
    else:
        sqlite_insert_query = """INSERT INTO KeyboardInput
                              (user, key, keyEvent, Time) 
                              VALUES (?, ?, ?, ?);"""    
        cur.executemany(sqlite_insert_query, eventList)
        con.commit()
    print("done")

def getKeyStroke(self):
    
    keyLogger = KeyLogger()
    hookManager = pyHook.HookManager() #Stworzenie hookManager- rejestruje i zarządza zdarzeniami 
    hookManager.KeyDown = keyLogger.keyDownEvent #Wykrycie zdarzenia keyDown-uruchomienie przypisanego KeyEvent Keyloggera
    hookManager.KeyUp = keyLogger.keyUpEvent #Wykrycie zdarzenia keyUp-uruchomienie przypisanego KeyEvent Keyloggera
    hookManager.HookKeyboard() #Oczekuj na zdarzenia z użyciem klawiatury

    keyLogger.mainLoop()
    # Unhooks the keyboard, no more data recorded, returns to menu
    hookManager.UnhookKeyboard()

In [5]:
class KeyLogger:
    def __init__(self):
        self.enterPressed = False
        self.eventList = []
        self.isCaps = False
        #self.message = ""
        
    def keyDownEvent(self, event):
        if event.KeyID == 20 and self.isCaps == False: 
            self.isCaps = True
        elif event.KeyID == 20 and self.isCaps == True: 
            self.isCaps = False     
        if event.KeyID>= 48 and event.KeyID<=57:  #Obsługa cyfr 
            event.Ascii = event.KeyID 
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122: #isCaps+ mała litera=duża litera
            event.Ascii = event.KeyID
        self.storeEvent("Down", event) 
        return True
        

    def keyUpEvent(self, event): 
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        print(chr(event.Ascii),end='')
        self.storeEvent("Up", event)
        return True

    def mainLoop(self):
        while not self.enterPressed:
            pythoncom.PumpWaitingMessages()

    def storeEvent(self, activity, event):
        keystrokeTime = int(event.Time) #pobranie czasu zdarzenia
        self.eventList.append ((userName,event.Ascii,activity, int(keystrokeTime)))

        # Użyj ESC do zatwierdzenia wpisywanego zdania 
        # Enter Key - KeyCode: 13 Ascii: 13 ScanCode: 28 - ESC = 27 @ Ascii
        if event.Ascii == 27:
            self.enterPressed = True
            userRecordData(self.eventList)

class App(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.pack()
        self._frame = None
        self.switch_frame(MainPage)

    def switch_frame(self, frame_class):
        new_frame = frame_class(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()
            
class MainPage(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master, bg='LightSkyBlue3')
        tk.Label(self, text='Weryfikacja', font=('bold', 48), bg='LightSkyBlue3').pack(side="top", fill="x", pady=10)
        tk.Button(self, text="Dodaj użytkownika", font=('bold', 24), bg='LightSkyBlue2',
                  command=lambda: master.switch_frame(AddUser)).pack(pady=10)
        tk.Button(self, text="Zweryfikuj użytkownika", font=('bold', 24), bg='LightSkyBlue2',
                  command=lambda: master.switch_frame(Verification)).pack(pady=10)  
            
            
class AddUser(tk.Frame):
    def __init__(self, master):
        global ver
        ver = False
        tk.Frame.__init__(self, master, bg='LightSkyBlue3')
        tk.Label(self, text='Dodanie użytkownika', font=('bold', 48), bg='LightSkyBlue3').pack(side="top", fill="x", pady=10)
        tk.Label(self, text='Podaj nazwę użytkownika:', font=('bold', 14), bg='LightSkyBlue3').pack()
        self.text_box = tk.Text(self, width=40, height=1)
        self.text_box.pack(pady=10)
        tk.Button(self, text="Zatwierdź", font=('bold', 24), bg='LightSkyBlue2',
                  command=lambda: self.sendUserName()).pack(pady=10)
        text_box2 = tk.Text(self, width=40, height=1)
        text_box2.pack(pady=10)
        text_box2.bind("<Key>", getKeyStroke)
        
        tk.Button(self, text="Powrót do strony głównej", font=('bold', 24), bg='LightSkyBlue2',
                  command=lambda: master.switch_frame(MainPage)).pack(pady=10)
        
    def sendUserName(self):
        global userName
        userName = self.text_box.get('1.0', 'end').rstrip()
        print(userName)
        
class Verification(tk.Frame):
    def __init__(self, master):
        global ver
        ver = True
        tk.Frame.__init__(self, master, bg='LightSkyBlue3')
        tk.Label(self, text='Zweryfikowanie użytkownika', font=('bold', 48), bg='LightSkyBlue3').pack(side="top", fill="x", pady=10)
        tk.Label(self, text='Podaj nazwę użytkownika:', font=('bold', 14), bg='LightSkyBlue3').pack()
        self.text_box = tk.Text(self, width=40, height=1)
        self.text_box.pack(pady=10)
        tk.Button(self, text="Zatwierdź", font=('bold', 24), bg='LightSkyBlue2',
                  command=lambda: self.sendUserName()).pack(pady=10)
        text_box2 = tk.Text(self, width=40, height=1)
        text_box2.pack(pady=10)
        text_box2.bind("<Key>", getKeyStroke)
        
        tk.Button(self, text="Powrót do strony głównej", font=('bold', 24), bg='LightSkyBlue2',
                  command=lambda: master.switch_frame(MainPage)).pack(pady=10)
        
    def sendUserName(self):
        global userName
        userName = self.text_box.get('1.0', 'end').rstrip()
        print(userName)


In [ ]:
myapp = App()
myapp.master.title("App")
myapp.master.minsize(50, 50)
myapp.mainloop()

lol
t o ja zweryfikuj mniedone
done
